In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col, when, unix_timestamp

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("ReadParquetFiles") \
    .config("spark.jars", "/home/xiaoyu/mariadb-java-client-3.1.2.jar") \
    .config("spark.driver.memory", "32G") \
    .config("spark.executor.memory", "8G") \
    .getOrCreate()

Picked up _JAVA_OPTIONS: -Xmx26624m
Picked up _JAVA_OPTIONS: -Xmx26624m


23/08/09 15:57:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
prom_file_path = "models/prom_pure_label.parquet"

In [4]:
data = spark.read.parquet(prom_file_path, header=True, inferSchema=True)

In [9]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import col, lag, when, unix_timestamp, sum
from pyspark.sql import Window
from pyspark.sql.functions import col, lag, when, unix_timestamp, sum, count

window = Window.partitionBy('node').orderBy('timestamp')

data = data.withColumn("time_difference", unix_timestamp(col('timestamp')) - unix_timestamp(lag('timestamp').over(window)))

data = data.withColumn("new_block", when(col("time_difference") > 30, 1).otherwise(0))

window2 = Window.partitionBy('node').orderBy('timestamp').rowsBetween(Window.unboundedPreceding, 0)
data = data.withColumn("block_id", F.sum("new_block").over(window2))

data = data.withColumn('row', F.row_number().over(window))

grouped_data = data.groupBy("node", "block_id")

grouped_data = grouped_data.agg(
    F.min("timestamp").alias("start_time"), 
    F.max("timestamp").alias("end_time"), 
    F.min("row").alias("start_row"), 
    F.max("row").alias("end_row"), 
    F.sum("isAnomaly").alias("anomalyCount"),  
    F.count("timestamp").alias("num_records")  
)
grouped_data = grouped_data.withColumn("block_duration_sec", unix_timestamp(col('end_time')) - unix_timestamp(col('start_time')))

grouped_data = grouped_data.withColumn(
    "block_duration",
    F.concat(
        F.expr('FLOOR(block_duration_sec / 3600)'), 
        F.lit(':'),
        F.format_string("%02d", F.expr('FLOOR((block_duration_sec % 3600) / 60)')), 
        F.lit(':'),
        F.format_string("%02d", F.expr('(block_duration_sec % 60)'))
    )
)

grouped_data.show()


+------+--------+-------------------+-------------------+---------+-------+------------+-----------+------------------+--------------+
|  node|block_id|         start_time|           end_time|start_row|end_row|anomalyCount|num_records|block_duration_sec|block_duration|
+------+--------+-------------------+-------------------+---------+-------+------------+-----------+------------------+--------------+
|r11n15|       0|2022-06-30 18:00:30|2022-07-07 13:32:00|        1|  19624|           0|      19624|            588690|     163:31:30|
|r11n15|       1|2022-07-07 13:36:00|2022-07-14 09:16:30|    19625|  39266|         264|      19642|            589230|     163:40:30|
|r11n15|       2|2022-07-14 11:17:00|2022-07-15 11:16:30|    39267|  42146|        1268|       2880|             86370|      23:59:30|
|r11n15|       3|2022-07-15 13:17:00|2022-07-16 11:16:30|    42147|  44786|           0|       2640|             79170|      21:59:30|
|r11n15|       4|2022-07-16 13:17:00|2022-07-17 11:16:3

In [11]:
grouped_data.orderBy(F.col("anomalyCount").desc()).show(30)

+------+--------+-------------------+-------------------+---------+-------+------------+-----------+------------------+--------------+
|  node|block_id|         start_time|           end_time|start_row|end_row|anomalyCount|num_records|block_duration_sec|block_duration|
+------+--------+-------------------+-------------------+---------+-------+------------+-----------+------------------+--------------+
|r14n20|      25|2022-08-26 22:10:00|2022-09-04 16:32:00|   158794| 184038|       25245|      25245|            757320|     210:22:00|
|r25n13|      32|2022-10-04 19:20:30|2022-10-17 16:25:00|   269160| 306249|       24053|      37090|           1112670|     309:04:30|
|r25n12|      38|2022-10-05 21:20:30|2022-10-17 16:18:30|   271314| 305270|       17507|      33957|           1018680|     282:58:00|
|r13n10|      38|2022-10-20 00:21:30|2022-10-25 12:32:30|   311752| 327614|       15794|      15863|            475860|     132:11:00|
|r26n21|      33|2022-10-04 19:20:30|2022-10-17 16:26:3

In [13]:
grouped_data.write.parquet('time_block_info.parquet')

In [9]:
data.iloc[50064:50067]

,id,timestamp,node,node_time_seconds,node_load15,surfsara_power_usage,up,node_netstat_Tcp_OutSegs,node_netstat_Tcp_InErrs,node_context_switches_total,...,FAILED,OUT_OF_MEMORY,NODE_FAIL,jobCount_30s,TIMEOUT_30s,COMPLETED_30s,CANCELLED_30s,FAILED_30s,OUT_OF_MEMORY_30s,NODE_FAIL_30s
63548751,13364309,2022-07-18 09:16:00,r11n5,1.658140e+09,0.07,36.0,1.0,408085000.0,0.0,3.334280e+10,...,0,0,0,0,0,0,0,0,0,0
63549031,13364310,2022-07-18 09:16:30,r11n5,1.658140e+09,0.07,36.0,1.0,408085000.0,0.0,3.334280e+10,...,0,0,0,0,0,0,0,0,0,0
63549311,14100874,2022-07-18 11:17:00,r11n5,1.658140e+09,0.00,36.0,1.0,408134000.0,0.0,3.334480e+10,...,0,0,0,0,0,0,0,0,0,0


In [22]:
longest_continuous_df.to_parquet('integrated_r11n5_continuous_data.parquet')

In [9]:
data.iloc[39264:39268]

,id,timestamp,node,node_time_seconds,node_load15,surfsara_power_usage,up,node_netstat_Tcp_OutSegs,node_netstat_Tcp_InErrs,node_context_switches_total,...,nvidia_gpu_duty_cycle-mean,nvidia_gpu_duty_cycle-max,node_network_transmit_packets_total-sum,node_udp_queues-sum,node_network_receive_bytes_total-sum,node_network_receive_packets_total-sum,node_network_receive_multicast_total-sum,node_disk_io_now-sum,node_rapl_package_joules_total-sum,node_network_receive_drop_total-sum
63629111,106559,2022-07-14 07:16:00,r11n5,1.657790e+09,15.81,196.0,1.0,272220000.0,0.0,3.304230e+10,...,NaN,NaN,272318983.0,0.0,5.797927e+11,224978338.0,2.0,0.0,192642.01,0.0
63629391,106560,2022-07-14 07:16:30,r11n5,1.657790e+09,15.73,164.0,1.0,272326000.0,0.0,3.304240e+10,...,NaN,NaN,272425215.0,0.0,5.799654e+11,225061969.0,2.0,0.0,196113.78,0.0
63629671,11082688,2022-07-14 09:17:00,r11n5,1.657790e+09,15.74,164.0,1.0,272430000.0,0.0,3.304260e+10,...,NaN,NaN,272529018.0,0.0,5.801070e+11,225142335.0,2.0,0.0,199597.53,0.0
63629951,11082689,2022-07-14 09:17:30,r11n5,1.657790e+09,15.75,164.0,1.0,272540000.0,0.0,3.304280e+10,...,NaN,NaN,272638589.0,0.0,5.802677e+11,225226486.0,2.0,0.0,203035.84,0.0
